In [1]:
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np

import re
import string

# Outline
1. Clean Tier List Table
    - Encoding tier list - (S+ = 0 -> D = 5)
    - Turn rates into float dtype
2. Clean Champion Strategies Table 
    - Manually edit to add missing champions and tips
    - Clean format
3. Sync champion names across ALL tables
4. Generate Tables for Features

## 0. Load Data

In [2]:
preprocessed_data_dir = './preprocessed_data'
processed_data_dir = './processed_data'

In [3]:
df_tier_list = pd.read_csv(f'{preprocessed_data_dir}/ugg_tier_list.csv')
df_champ_roles = pd.read_csv(f'{preprocessed_data_dir}/wiki_champ_roles.csv')
df_champ_strats = pd.read_csv(f'{preprocessed_data_dir}/tencent_champion_strats_clean.csv')
df_champ_details = pd.read_csv(f'{preprocessed_data_dir}/wiki_champion_details.csv')

## 1. Clean Tier List Table

In [4]:
def convert_to_float(percent):
    if isinstance(percent,str):
        return round(float(percent.replace('%',''))/100,4)
    else:
        return float('nan')

In [5]:
# df_tier_list['win_rate'] = df_tier_list['win_rate'].apply(convert_to_float)
# df_tier_list['pick_rate'] = df_tier_list['pick_rate'].apply(convert_to_float)
# df_tier_list['ban_rate'] = df_tier_list['ban_rate'].apply(convert_to_float)

In [6]:
# tier_mapping = {'S+':0,'S':1,'A':2,'B':3,'C':4,'D':5}
# df_tier_list['tier'] = df_tier_list['tier'].map(tier_mapping)

## 2. Clean Champion Strategies Table

In [7]:
def clean_paragraphs(txt):
    segments = pd.Series(txt.split('.')).apply(lambda x: x.replace('-','').strip()+'.')[:-1]
    return '\n'.join(segments).strip()

In [8]:
df_champ_strats['play_strats'] = df_champ_strats['play_strats'].apply(clean_paragraphs)
df_champ_strats['counter_strats'] = df_champ_strats['counter_strats'].apply(clean_paragraphs)

In [9]:
df_champ_strats['champion'] = df_champ_strats['champion'].apply(lambda x: x.replace(',','').replace('’',"'"))

In [10]:
df_champ_strats.drop('champion_cn',axis=1,inplace=True)

## 3. Sync champion names across ALL tables

In [11]:
tier_list_set = set(df_tier_list['champion'].unique())
champ_roles_set = set(df_champ_roles['champion'].unique())
champ_strats_set = set(df_champ_strats['champion'].unique())
champ_details_set = set(df_champ_details['champion'].unique())

In [12]:
tier_list_set == champ_roles_set

True

In [13]:
tier_list_set == champ_strats_set

True

In [14]:
tier_list_set == champ_details_set

True

## 4. Generate Tables for Features

### 4.1 Feature 1
- Champion statistics table
    - Division
    - Champion
    - Lane
    - Tier
    - Win rate
    - Pick rate
    - Ban rate
    - Class
    - Secondary class
    - Role
    - Counter picks
    - Matches
- Main table: Tier List Table
- Join [Class,Secondary class] from the Champion Details Table by champion name
- Join [Role] from Champion Roles Table by champion name

In [15]:
df_champ_stats = pd.merge(df_tier_list,df_champ_roles[['champion','primary_role']],how='left',on='champion')
df_champ_stats = pd.merge(df_champ_stats,df_champ_details[['champion','primary_class','secondary_class']],how='left',on='champion')
df_champ_stats = df_champ_stats[['divison', 'lane', 'champion', 'tier', 'win_rate', 'pick_rate',
                                 'ban_rate', 'primary_role', 'primary_class',
                                 'secondary_class','counters','matches']]

In [16]:
df_champ_stats.head()

,divison,lane,champion,tier,win_rate,pick_rate,ban_rate,primary_role,primary_class,secondary_class,counters,matches
0,Iron,Jungle,Karthus,2,0.6818,0.010,0.003,Battlemage,Mage,NaN,"Malphite,Warwick,Hecarim,Rammus,Vi,Udyr,Lee Sin",66
1,Iron,Jungle,Poppy,2,0.6522,0.007,0.004,Warden,Tank,Fighter,"Sion,Rengar,Kayn,Kindred,Ivern,Amumu,Olaf",46
2,Iron,Supp,Shen,2,0.6500,0.006,0.008,Warden,Tank,NaN,"Swain,Blitzcrank,Maokai,Kayn,Zoe,Seraphine,Rakan",40
3,Iron,Top,Warwick,2,0.6182,0.008,0.102,Diver,Fighter,Tank,"Jax,Malphite,Gangplank,Aatrox,Vayne,Dr. Mundo,...",55
4,Iron,Mid,Sett,2,0.5962,0.008,0.108,Juggernaut,Fighter,Tank,"Irelia,Nocturne,Heimerdinger,Pantheon,Lucian,S...",52


In [17]:
df_champ_stats.to_csv(f'{processed_data_dir}/champion_stats.csv',index=False)

## 4.2 Feature 2
- Champion properties table
    - champion
    - class
    - ratings
    - role

In [18]:
df_champ_properties = pd.merge(df_champ_details,df_champ_roles[['champion','primary_role']],how='left',on='champion')

In [19]:
df_champ_properties.rename(columns={'primary_role':'role'},inplace=True)

In [20]:
def merge_classes(class1,class2):
    if isinstance(class2,str):
        return f'{class1}/{class2}'
    else:
        return class1

In [21]:
df_champ_properties['class'] = df_champ_properties.apply(lambda x: merge_classes(x['primary_class'],x['secondary_class']),axis=1)
df_champ_properties = df_champ_properties[['champion','class','role','ratings']]

In [22]:
df_champ_properties.head()

,champion,class,role,ratings
0,Aatrox,Fighter/Tank,Juggernaut,"{'Physical Damage': 3, 'Toughness': 3, 'Contro..."
1,Ahri,Mage/Assassin,Burst,"{'Magic Damage': 3, 'Toughness': 1, 'Control':..."
2,Akali,Assassin,Assassin,"{'Physical Damage': 3, 'Toughness': 1, 'Contro..."
3,Alistar,Tank/Support,Vanguard,"{'Magic Damage': 1, 'Toughness': 3, 'Control':..."
4,Amumu,Tank/Mage,Vanguard,"{'Magic Damage': 2, 'Toughness': 3, 'Control':..."


In [23]:
df_champ_properties.to_csv(f'{processed_data_dir}/champion_properties.csv',index=False)

## 5. Upload to S3

In [26]:
from utility.s3_file_transfer import upload_df_to_s3

In [28]:
upload_df_to_s3(df_champ_stats,'peter-ff15-data/champion_stats.csv')
upload_df_to_s3(df_champ_properties,'peter-ff15-data/champion_properties.csv')